In [1]:
!pip install nltk tensorflow


  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for tensorboard<2.17,>=2.16 from https://files.pythonhosted.org/packages/3a/d0/b97889ffa769e2d1fdebb632084d5e8b53fc299d43a537acee7ec0c021a3/tensorboard-2.16.2-py3-none-any.whl.metadata
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl (377.0 MB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-i

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\DELL\\anaconda3\\Lib\\site-packages\\~-nsorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
paragraph = """
Artificial intelligence is revolutionizing the world. Machine learning is a crucial component of AI. 
Natural language processing helps computers understand human language. Neural networks are designed to mimic the human brain. 
Deep learning has advanced the field of AI significantly. With AI, automation is becoming a reality. 
Data science relies heavily on algorithms and statistical models. AI applications are widespread in industries like healthcare, finance, and education.
"""


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

# Tokenizing the paragraph into sentences and words
sentences = nltk.sent_tokenize(paragraph)
stop_words = set(stopwords.words('english'))

# Tokenizing and removing stopwords
def preprocess_text(sentences):
    words = []
    for sentence in sentences:
        tokenized_sentence = word_tokenize(sentence.lower())
        filtered_sentence = [w for w in tokenized_sentence if w.isalnum() and w not in stop_words]
        words.extend(filtered_sentence)
    return words

words = preprocess_text(sentences)
print("Vocabulary Size:", len(set(words)))
print("Tokenized Words:", words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Vocabulary Size: 42
Tokenized Words: ['artificial', 'intelligence', 'revolutionizing', 'world', 'machine', 'learning', 'crucial', 'component', 'ai', 'natural', 'language', 'processing', 'helps', 'computers', 'understand', 'human', 'language', 'neural', 'networks', 'designed', 'mimic', 'human', 'brain', 'deep', 'learning', 'advanced', 'field', 'ai', 'significantly', 'ai', 'automation', 'becoming', 'reality', 'data', 'science', 'relies', 'heavily', 'algorithms', 'statistical', 'models', 'ai', 'applications', 'widespread', 'industries', 'like', 'healthcare', 'finance', 'education']


In [4]:
# Creating the context and target pairs
def generate_context_target_pairs(words, window_size=2):
    context_target_pairs = []
    for i in range(window_size, len(words) - window_size):
        context = words[i-window_size:i] + words[i+1:i+1+window_size]
        target = words[i]
        context_target_pairs.append((context, target))
    return context_target_pairs

context_target_pairs = generate_context_target_pairs(words)
print("Sample Context-Target Pairs:", context_target_pairs[:5])


Sample Context-Target Pairs: [(['artificial', 'intelligence', 'world', 'machine'], 'revolutionizing'), (['intelligence', 'revolutionizing', 'machine', 'learning'], 'world'), (['revolutionizing', 'world', 'learning', 'crucial'], 'machine'), (['world', 'machine', 'crucial', 'component'], 'learning'), (['machine', 'learning', 'component', 'ai'], 'crucial')]


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Lambda
from tensorflow.keras.models import Sequential

# Define the CBOW model
vocab_size = 100  # Example vocab size
embedding_dim = 10  # Embedding dimensions

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=4))  # 2 context words on each side
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))  # Take average of the context words' embeddings
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)